# 1 - Business Problem

In [1]:
import pandas as pd

df = pd.read_csv("./Data/machine_dataset_train_data.csv")
df.head()

,VendorName,ModelName,MYCT,MMIN,MMAX,CACH,CHMIN,CHMAX,PRP,ERP
0,adviser,32/60,125,256,6000,256,16,128,198,199
1,amdahl,470v/7,29,8000,32000,32,8,32,269,253
2,amdahl,470v/7a,29,8000,32000,32,8,32,220,253
3,amdahl,470v/7b,29,8000,32000,32,8,32,172,253
4,amdahl,470v/7c,29,8000,16000,32,8,16,132,132


In [2]:
df.shape

(189, 10)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189 entries, 0 to 188
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   VendorName  189 non-null    object
 1   ModelName   189 non-null    object
 2   MYCT        189 non-null    int64 
 3   MMIN        189 non-null    int64 
 4   MMAX        189 non-null    int64 
 5   CACH        189 non-null    int64 
 6   CHMIN       189 non-null    int64 
 7   CHMAX       189 non-null    int64 
 8   PRP         189 non-null    int64 
 9   ERP         189 non-null    int64 
dtypes: int64(8), object(2)
memory usage: 14.9+ KB


# 2 - Connect to Azure Machine Learning Workspace

In [4]:
# Import required libraries
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import automl, Input, command

In [7]:
# Configure workspace details and get a handle to workspace

# Details of azure ML workspace
subscription_id='your_subscription_id'
resource_group='you_resource_group'
workspace = "automl-ws12"

# Get a handle to workspace
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

In [8]:
# Show Azure ML Workspace information

workspace = ml_client.workspaces.get(name=ml_client.workspace_name)

subscription_id = ml_client.connections._subscription_id
resource_group = workspace.resource_group
workspace_name = ml_client.workspace_name

output = {}
output["Workspace"] = workspace_name
output["Subscription ID"] = subscription_id
output["Resource Group"] = resource_group
output["Location"] = workspace.location
output

{'Workspace': 'automl-ws12',
 'Subscription ID': '3b57d2fe-08b1-4fe9-b535-f5c4387b9a66',
 'Resource Group': 'automl-rg12',
 'Location': 'westus'}

# 3 - MLTable with input Training Data

In [9]:
# Create mltable

import mltable

paths = [{"file": "./data/training-mltable-folder/machine_dataset_train_data.csv"}]

train_table = mltable.from_delimited_files(paths)
train_table.save("./data/training-mltable-folder")

In [10]:
# Create mltable data input

my_training_data_input = Input(type=AssetTypes.MLTABLE, path="./data/training-mltable-folder")

# 4 - Compute target setup

In [11]:
from azure.ai.ml.entities import AmlCompute
from azure.core.exceptions import ResourceNotFoundError

compute_name = "cpu-cluster"

try:
    _ = ml_client.compute.get(compute_name)
    print("Found existing compute target.")
except ResourceNotFoundError:
    print("Creating a new compute target...")
    compute_config = AmlCompute(
        name=compute_name,
        type="amlcompute",
        size="STANDARD_DS12_V2",
        idle_time_before_scale_down=120,
        min_instances=0,
        max_instances=6,
    )
    ml_client.begin_create_or_update(compute_config).result()

Creating a new compute target...


# 5 - Configure and run AutoML Regression training job

In [13]:
max_trials = 5
exp_name = "regression-experiment"

In [14]:
# AutoML regression job

regression_job = automl.regression(
    experiment_name=exp_name,
    training_data=my_training_data_input,
    target_column_name='ERP',
    primary_metric="R2Score",
    n_cross_validations=5,
    enable_model_explainability=True,
    tags={"my custom key": "my custom value"}
)

# Limits
regression_job.set_limits(
    timeout_minutes=60,
    trial_timeout_minutes=4,
    max_trials=max_trials,
    enable_early_termination=True
)

In [15]:
# Submit the job run

returned_job = ml_client.jobs.create_or_update(regression_job)
print(f"Created Job: {returned_job}")

Uploading training-mltable-folder (0.01 MBs): 100%|██████████| 8579/8579 [00:00<00:00, 69627.09it/s]




Created Job: compute: azureml:/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/resourceGroups/automl-rg12/providers/Microsoft.MachineLearningServices/workspaces/automl-ws12/computes/
creation_context:
  created_at: '2024-04-28T19:19:08.099146+00:00'
  created_by: Vijay Gadhave
  created_by_type: User
display_name: olive_fennel_dct48dckcj
experiment_name: regression-experiment
id: azureml:/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/resourceGroups/automl-rg12/providers/Microsoft.MachineLearningServices/workspaces/automl-ws12/jobs/olive_fennel_dct48dckcj
limits:
  enable_early_termination: true
  max_concurrent_trials: 1
  max_cores_per_trial: -1
  max_nodes: 1
  max_trials: 5
  timeout_minutes: 60
  trial_timeout_minutes: 4
log_verbosity: info
n_cross_validations: 5
name: olive_fennel_dct48dckcj
outputs: {}
primary_metric: r2_score
properties: {}
resources:
  instance_count: 1
  shm_size: 2g
services:
  Studio:
    endpoint: https://ml.azure.com/runs/olive_fennel_dct48dckcj?ws

In [16]:
# Wait for job to complete and stream updates

ml_client.jobs.stream(returned_job.name)

RunId: olive_fennel_dct48dckcj
Web View: https://ml.azure.com/runs/olive_fennel_dct48dckcj?wsid=/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/resourcegroups/automl-rg12/workspaces/automl-ws12

Execution Summary
RunId: olive_fennel_dct48dckcj
Web View: https://ml.azure.com/runs/olive_fennel_dct48dckcj?wsid=/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/resourcegroups/automl-rg12/workspaces/automl-ws12



In [17]:
# Get a URL for the status of the job
returned_job.services["Studio"].endpoint

'https://ml.azure.com/runs/olive_fennel_dct48dckcj?wsid=/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/resourcegroups/automl-rg12/workspaces/automl-ws12&tid=843e1e65-e875-4fbd-8ddc-ce696f3a551d'

In [18]:
returned_job.name

'olive_fennel_dct48dckcj'

# 6 - Retrieve Best Trial

### Obtain tracking URI for MLFlow

In [19]:
# Obtain the tracking URI from MLClient

import mlflow

MLFLOW_TRACKING_URI = ml_client.workspaces.get(name=ml_client.workspace_name).mlflow_tracking_uri
print(MLFLOW_TRACKING_URI)

azureml://westus.api.azureml.ms/mlflow/v1.0/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/resourceGroups/automl-rg12/providers/Microsoft.MachineLearningServices/workspaces/automl-ws12


In [20]:
# Set the MLFLOW TRACKING URI

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
print("\nCurrent tracking uri: {}".format(mlflow.get_tracking_uri()))


Current tracking uri: azureml://westus.api.azureml.ms/mlflow/v1.0/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/resourceGroups/automl-rg12/providers/Microsoft.MachineLearningServices/workspaces/automl-ws12


In [21]:
# Initialize MLFlow client

from mlflow.tracking.client import MlflowClient
from mlflow.artifacts import download_artifacts


mlflow_client = MlflowClient()

### Get AutoML parent Job

In [22]:
job_name = returned_job.name

# Get the parent run
mlflow_parent_run = mlflow_client.get_run(job_name)

print("Parent Run: ")
print(mlflow_parent_run)

Parent Run: 
<Run: data=<RunData: metrics={'explained_variance': 0.8920387935716176,
 'mean_absolute_error': 13.539137179346207,
 'mean_absolute_percentage_error': 8.872139228421817,
 'median_absolute_error': 2.567427062988281,
 'normalized_mean_absolute_error': 0.011070431054248736,
 'normalized_median_absolute_error': 0.002099286233023942,
 'normalized_root_mean_squared_error': 0.034096838205383236,
 'normalized_root_mean_squared_log_error': 0.03225134363934273,
 'r2_score': 0.8877571661083964,
 'root_mean_squared_error': 41.70043312518369,
 'root_mean_squared_log_error': 0.14027628901087677,
 'spearman_correlation': 0.9849634259945527}, params={}, tags={'_azureml.ComputeTargetType': '',
 'automl_best_child_run_id': 'olive_fennel_dct48dckcj_1',
 'fit_time_000': '0.0226334;0.0766718;0.05308700000000001;2;2',
 'iteration_000': '0;1;2;3;4',
 'mlflow.rootRunId': 'olive_fennel_dct48dckcj',
 'mlflow.runName': 'olive_fennel_dct48dckcj',
 'mlflow.user': 'Vijay Gadhave',
 'model_explain_best_

In [23]:
# Print parent run tags. 'automl_best_child_run_id' tag should be there.
print(mlflow_parent_run.data.tags)

{'my custom key': 'my custom value', 'model_explain_run': 'best_run', '_azureml.ComputeTargetType': '', 'pipeline_id_000': 'faf12f74cf9bbd358ca5525682c5030d36f7be7c;4bc4ec47eb8df2d5d68b361cd60120e65196f757;5cc37daec73ea64276ef956449645cdb519fdfc6;__AutoML_Ensemble__;__AutoML_Stack_Ensemble__', 'score_000': '0.6359901279465644;0.8877571661083964;-0.4691851490369148;0.8877571661083964;0.8100906326416171', 'predicted_cost_000': '0;0;0.5;0;0', 'fit_time_000': '0.0226334;0.0766718;0.05308700000000001;2;2', 'training_percent_000': '100;100;100;100;100', 'iteration_000': '0;1;2;3;4', 'run_preprocessor_000': 'MaxAbsScaler;MaxAbsScaler;StandardScalerWrapper;;', 'run_algorithm_000': 'LightGBM;XGBoostRegressor;XGBoostRegressor;VotingEnsemble;StackEnsemble', 'automl_best_child_run_id': 'olive_fennel_dct48dckcj_1', 'model_explain_best_run_child_id': 'olive_fennel_dct48dckcj_1', 'mlflow.rootRunId': 'olive_fennel_dct48dckcj', 'mlflow.runName': 'olive_fennel_dct48dckcj', 'mlflow.user': 'Vijay Gadhave'

### Get AutoML best child run

In [ ]:
# Get the best model's child run

best_child_run_id = mlflow_parent_run.data.tags["automl_best_child_run_id"]
print("Found best child run id: ", best_child_run_id)

best_run = mlflow_client.get_run(best_child_run_id)

print("Best child run: ")
print(best_run)

In [26]:
# Get best model run's metrics
best_run.data.metrics

{'r2_score': 0.8877571661083964,
 'normalized_mean_absolute_error': 0.011070431054248736,
 'mean_absolute_error': 13.539137179346207,
 'mean_absolute_percentage_error': 8.872139228421817,
 'normalized_root_mean_squared_log_error': 0.03225134363934273,
 'spearman_correlation': 0.9849634259945527,
 'explained_variance': 0.8920387935716176,
 'root_mean_squared_log_error': 0.14027628901087677,
 'root_mean_squared_error': 41.70043312518369,
 'normalized_root_mean_squared_error': 0.034096838205383236,
 'median_absolute_error': 2.567427062988281,
 'normalized_median_absolute_error': 0.002099286233023942}

### Download best model locally

In [27]:
import os

# Create local folder
local_dir = "./artifact_downloads"
if not os.path.exists(local_dir):
    os.mkdir(local_dir)

In [28]:
# Download run's artifacts/outputs
local_path = download_artifacts(run_id=best_run.info.run_id, artifact_path="outputs", dst_path=local_dir)
print("Artifacts downloaded in: {}".format(local_path))
print("Artifacts: {}".format(os.listdir(local_path)))

Artifacts downloaded in: /mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-instance01/code/Users/vijaygadhave199/artifact_downloads/outputs
Artifacts: ['conda_env_v_1_0_0.yml', 'engineered_feature_names.json', 'env_dependencies.json', 'featurization_summary.json', 'generated_code', 'internal_cross_validated_models.pkl', 'mlflow-model', 'model.pkl', 'pipeline_graph.json', 'run_id.txt', 'scoring_file_pbi_v_1_0_0.py', 'scoring_file_v_1_0_0.py', 'scoring_file_v_2_0_0.py']


In [29]:
# Show the contents of MLFlow model folder
os.listdir("./artifact_downloads/outputs/mlflow-model")

['.amlignore',
 '.amlignore.amltmp',
 'conda.yaml',
 'MLmodel',
 'model.pkl',
 'python_env.yaml',
 'requirements.txt']

# 7 - Create managed online endpoint

In [30]:
# import required libraries
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
    ProbeSettings,
)
from azure.ai.ml.constants import ModelType

In [31]:
# Creating an unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "regression-" + datetime.datetime.now().strftime("%m%d%H%M%f")
print(online_endpoint_name)

regression-04282028824106


In [32]:
# create an online endpoint

endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is a sample online endpoint for mlflow model",
    auth_mode="key",
    tags={"foo": "bar"},
)

In [33]:
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://regression-04282028824106.westus.inference.ml.azure.com/score', 'openapi_uri': 'https://regression-04282028824106.westus.inference.ml.azure.com/swagger.json', 'name': 'regression-04282028824106', 'description': 'this is a sample online endpoint for mlflow model', 'tags': {'foo': 'bar'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/resourcegroups/automl-rg12/providers/microsoft.machinelearningservices/workspaces/automl-ws12/onlineendpoints/regression-04282028824106', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/providers/Microsoft.MachineLearningServices/locations/westus/mfeOperationsStatus/oe:37ec976d-d4c6-40eb-a7f5-e15ebf372a4d:16f0d4f1-3ba3-4e4a-80f8-fdd29ce2b4a3?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/3b57d2fe-08b1-4f

# 8 - Register best model

In [34]:
model_name = "hardware-performance-model"

model = Model(
    path=f"azureml://jobs/{best_run.info.run_id}/outputs/artifacts/outputs/mlflow-model/",
    name=model_name,
    description="my regression model",
    type=AssetTypes.MLFLOW_MODEL,
)

registered_model = ml_client.models.create_or_update(model)

In [35]:
registered_model.id

'/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/resourceGroups/automl-rg12/providers/Microsoft.MachineLearningServices/workspaces/automl-ws12/models/hardware-performance-model/versions/1'

# 9 - Deploy best model

In [39]:
deployment = ManagedOnlineDeployment(
    name="hardware-performance-deploy",
    endpoint_name=online_endpoint_name,
    model=registered_model.id,
    instance_type="Standard_DS2_V2",
    instance_count=1,
    liveness_probe=ProbeSettings(
        failure_threshold=30,
        success_threshold=1,
        timeout=2,
        period=10,
        initial_delay=2000,
    ),
    readiness_probe=ProbeSettings(
        failure_threshold=10,
        success_threshold=1,
        timeout=10,
        period=10,
        initial_delay=2000,
    ),
)

In [40]:
ml_client.online_deployments.begin_create_or_update(deployment).result()

Instance type Standard_DS2_V2 may be too small for compute resources. Minimum recommended compute SKU is Standard_DS3_v2 for general purpose endpoints. Learn more about SKUs here: https://learn.microsoft.com/en-us/azure/machine-learning/referencemanaged-online-endpoints-vm-sku-list
Check: endpoint regression-04282028824106 exists


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

ManagedOnlineDeployment({'private_network_connection': None, 'provisioning_state': 'Succeeded', 'endpoint_name': 'regression-04282028824106', 'type': 'Managed', 'name': 'hardware-performance-deploy', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/providers/Microsoft.MachineLearningServices/locations/westus/mfeOperationsStatus/od:37ec976d-d4c6-40eb-a7f5-e15ebf372a4d:aa8124c4-a235-401d-b9c2-115b3b33c4bf?api-version=2023-04-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/resourceGroups/automl-rg12/providers/Microsoft.MachineLearningServices/workspaces/automl-ws12/onlineEndpoints/regression-04282028824106/deployments/hardware-performance-deploy', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-instance01/code/Users/vijaygadhave199', 'creation_context': None, 'serialize': <msrest.seria

In [41]:
# existing traffic details
print(endpoint.traffic)

{}


In [42]:
# hardware performance deployment to take 100% traffic
endpoint.traffic = {"hardware-performance-deploy": 100}
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://regression-04282028824106.westus.inference.ml.azure.com/score', 'openapi_uri': 'https://regression-04282028824106.westus.inference.ml.azure.com/swagger.json', 'name': 'regression-04282028824106', 'description': 'this is a sample online endpoint for mlflow model', 'tags': {'foo': 'bar'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/resourcegroups/automl-rg12/providers/microsoft.machinelearningservices/workspaces/automl-ws12/onlineendpoints/regression-04282028824106', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/providers/Microsoft.MachineLearningServices/locations/westus/mfeOperationsStatus/oe:37ec976d-d4c6-40eb-a7f5-e15ebf372a4d:648b3e0a-e9c8-42ae-9cb0-430073b0bdb5?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/3b57d2fe-08b1-4f

In [43]:
# existing traffic details
print(endpoint.traffic)

{'hardware-performance-deploy': 100}


# 10 - Test the deployment

In [48]:
# test the deployment with sample data

import pandas as pd

test_data = pd.read_csv("./data/test-mltable-folder/machine_dataset_test_data.csv")
test_data = test_data.drop("ERP", axis=1)

# Convert Data to JSON format
test_data_json = test_data.to_json(orient="records", indent=4)
data = (
    '{ \
          "input_data": {"data": '
    + test_data_json
    + "}}"
)

#  Prepare Request File
request_file_name = "sample-request-hardware-performance.json"

with open(request_file_name, "w") as request_file:
    request_file.write(data)

# Invoke the Deployed Model Endpoint
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="hardware-performance-deploy",
    request_file="sample-request-hardware-performance.json",
)

'[198.94456481933594, 253.0386505126953, 22.967561721801758, 44.99810791015625, 28.005020141601562, 102.0025634765625, 30.003328323364258, 38.003334045410156, 33.990787506103516, 33.9858283996582, 156.9995574951172, 17.967273712158203, 52.994171142578125, 22.97707176208496, 20.012155532836914]'

In [49]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

{'hardware-performance-deploy': 100}
https://regression-04282028824106.westus.inference.ml.azure.com/score


In [50]:
# Delete the deployment and endpoint
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)

.................................................................

# 11 - Use auto generated model training code for retraining on new data

### Create Compute Target

In [51]:
# Create or Attach existing AmlCompute

from azure.ai.ml.entities import AmlCompute
from azure.core.exceptions import ResourceNotFoundError

compute_name = "cpu-cluster"

try:
    _ = ml_client.compute.get(compute_name)
    print("Found existing compute target.")
except ResourceNotFoundError:
    print("Creating a new compute target...")
    compute_config = AmlCompute(
        name=compute_name,
        type="amlcompute",
        size="STANDARD_DS3_V2",
        idle_time_before_scale_down=120,
        min_instances=0,
        max_instances=6,
    )
    ml_client.begin_create_or_update(compute_config).result()

Creating a new compute target...


### Register the dataset

In [52]:
# Create MLTable

import mltable

paths = [{'file': './data/retrained-mltable-folder/machine_dataset_retrain_data.csv'}]

retrain_table = mltable.from_delimited_files(paths)
retrain_table.save('./data/retrained-mltable-folder')

In [53]:
from azure.ai.ml.entities import Data

retrain_data = Data(
    path="./data/retrained-mltable-folder/",
    type=AssetTypes.MLTABLE,
    description="Updated training dataset",
    name="machine"
)

retrain_data = ml_client.data.create_or_update(retrain_data)

Uploading retrained-mltable-folder (0.01 MBs): 100%|██████████| 9423/9423 [00:00<00:00, 78717.91it/s]




In [54]:
import mlflow

# Obtain the tracking URI from MLClient
MLFLOW_TRACKING_URI = ml_client.workspaces.get(name=ml_client.workspace_name).mlflow_tracking_uri
print(MLFLOW_TRACKING_URI)

# Set the MLFLOW TRACKING URI
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

print("\nCurrent tracking uri: {}".format(mlflow.get_tracking_uri()))

azureml://westus.api.azureml.ms/mlflow/v1.0/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/resourceGroups/automl-rg12/providers/Microsoft.MachineLearningServices/workspaces/automl-ws12

Current tracking uri: azureml://westus.api.azureml.ms/mlflow/v1.0/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/resourceGroups/automl-rg12/providers/Microsoft.MachineLearningServices/workspaces/automl-ws12


### Download script

In [55]:
from mlflow.tracking.client import MlflowClient

# Initialize MLFlow client
mlflow_client = MlflowClient()

# Get the parent run
job_name = returned_job.name
mlflow_parent_run = mlflow_client.get_run(job_name)
print("Parent Run: ")
print(mlflow_parent_run)

# Get the best model's child run
best_child_run_id = mlflow_parent_run.data.tags["automl_best_child_run_id"]
print("Found best child run id: ", best_child_run_id)
best_run = mlflow_client.get_run(best_child_run_id)

# Download run's artifacts/outputs
local_dir = "./artifact_downloads/"
if not os.path.exists(local_dir):
    os.mkdir(local_dir)
local_path = download_artifacts(
    run_id=best_run.info.run_id, artifact_path="outputs", dst_path=local_dir
)
print("Artifacts downloaded in: {}".format(local_path))
print("Artifacts: {}".format(os.listdir(local_path)))

Parent Run: 
<Run: data=<RunData: metrics={'explained_variance': 0.8920387935716176,
 'mean_absolute_error': 13.539137179346207,
 'mean_absolute_percentage_error': 8.872139228421817,
 'median_absolute_error': 2.567427062988281,
 'normalized_mean_absolute_error': 0.011070431054248736,
 'normalized_median_absolute_error': 0.002099286233023942,
 'normalized_root_mean_squared_error': 0.034096838205383236,
 'normalized_root_mean_squared_log_error': 0.03225134363934273,
 'r2_score': 0.8877571661083964,
 'root_mean_squared_error': 41.70043312518369,
 'root_mean_squared_log_error': 0.14027628901087677,
 'spearman_correlation': 0.9849634259945527}, params={}, tags={'_azureml.ComputeTargetType': '',
 'automl_best_child_run_id': 'olive_fennel_dct48dckcj_1',
 'fit_time_000': '0.0226334;0.0766718;0.05308700000000001;2;2',
 'iteration_000': '0;1;2;3;4',
 'mlflow.rootRunId': 'olive_fennel_dct48dckcj',
 'mlflow.runName': 'olive_fennel_dct48dckcj',
 'mlflow.user': 'Vijay Gadhave',
 'model_explain_best_

In [56]:
# Find the curated environment used for training

import yaml

# constructing the path to the "MLmodel"
model_file = os.path.join(local_dir, "outputs", "mlflow-model", "MLmodel")

with open(model_file, "r") as model_stream:
    model_yaml = yaml.safe_load(model_stream)

    # extracting the name of the training environment from the model_yaml dictionary
    training_environment_name = (
        "AzureML-AutoML:" + model_yaml["metadata"]["azureml.base_image"].split(":")[-1]
    )
    print("Training emvironment {}".format(training_environment_name))

Training emvironment AzureML-AutoML:167


### Create and run command

In [57]:
# constructs a string that defines the command to be executed in the job
command_str = f"python script.py --training_dataset_uri {retrain_data.path}"

command_job = command(
    code="./artifact_downloads/outputs/generated_code/",
    command=command_str,
    tags=dict(automl_child_run_id=best_run.info.run_id),
    environment=training_environment_name,
    compute=compute_name,
    experiment_name=exp_name,
)

script_job = ml_client.create_or_update(command_job)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Uploading generated_code (0.02 MBs): 1